In [1]:
# !echo $RAYCLUSTER_AUTOSCALER_HEAD_SVC_PORT_10001_TCP_ADDR
# !echo $RAYCLUSTER_AUTOSCALER_HEAD_SVC_PORT_10001_TCP_PORT

In [2]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [3]:
import ray
import time

In [4]:
# from utils import logger

In [10]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from utils import helper, config, rayer
from ml.models.base.brisk_xgboost import BriskXGBoost

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common
from sklearn.preprocessing import StandardScaler


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
rayer.get_global_cluster(num_cpus=4)

In [7]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-24 02:08:32.811821 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 2 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/5.0 CPU
 0.00/16.764 GiB memory
 0.00/4.905 GiB object_store_memory

Demands:
 {'CPU': 1}: 4+ from request_resources()


In [8]:
# ray.autoscaler.sdk.request_resources(num_cpus=0)

In [11]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

ss = StandardScaler()
X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)


In [20]:

brisk_xgb = BriskXGBoost('brisk_xgb')
brisk_xgb.boosted_round = 100
brisk_xgb.n_trials = 5000


In [21]:
brisk_xgb.fetch_model()

2022-11-17T13:53:36CST : INFO : brisk_xgboost : __discover_model__ : 115 : Message : brisk_xgb: Starting training for trials:5000, boosted rounds: 100, max depth: 10
2022-11-17T13:59:05CST : INFO : brisk_xgboost : __discover_model__ : 125 : Message : brisk_xgb: Number of trials: 5000
2022-11-17T13:59:05CST : INFO : brisk_xgboost : __discover_model__ : 127 : Message : Best trial:4465
2022-11-17T13:59:05CST : INFO : brisk_xgboost : __discover_model__ : 129 : Message :   Params: 
2022-11-17T13:59:05CST : INFO : brisk_xgboost : __discover_model__ : 131 : Message :     booster gbtree
2022-11-17T13:59:05CST : INFO : brisk_xgboost : __discover_model__ : 131 : Message :     lambda 0.033295017028611644
2022-11-17T13:59:05CST : INFO : brisk_xgboost : __discover_model__ : 131 : Message :     alpha 0.013050765243725728
2022-11-17T13:59:05CST : INFO : brisk_xgboost : __discover_model__ : 131 : Message :     max_depth 1
2022-11-17T13:59:05CST : INFO : brisk_xgboost : __discover_model__ : 131 : Messa

In [16]:

slug_xgb = SlugXGBoost('slug_xgb', X_train_scalar, X_test_scalar, y_train, y_test)
slug_xgb.boosted_round = 50
slug_xgb.n_trials = 1000


In [17]:
slug_xgb.fetch_model()

2022-11-17T13:52:14CST : INFO : slug_xgboost : __discover_model__ : 168 : Message : slug_xgb: Starting training for trials:1000, boosted rounds: 50, max depth: 10
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 181 : Message : slug_xgb: Number of trials: 1000
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 183 : Message : Best trial:611
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 185 : Message :   Params: 
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     lambda 0.019399866419368653
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     alpha 0.0020906308336665715
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     eta 0.3727957591519319
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     gamma 0.0033926747144862554
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ :

In [36]:

slug_rf = SlugRF('slug_rf', X_train_scalar, X_test_scalar, y_train, y_test)
slug_rf.max_n_estimators = 5000
slug_rf.n_trials = 100
slug_rf.max_depth = 200

In [37]:
best_fit = slug_rf.fetch_model()

2022-11-17T14:56:36CST : INFO : slug_rf : __discover_model__ : 105 : Message : slug_rf: Starting training for trials:100, max depth: 200, max estimators 5000
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 122 : Message : slug_rf: Number of trials: 100
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 124 : Message : Best trial:74
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 126 : Message :   Params: 
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     n_estimators 62
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     max_depth 26
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     min_samples_split 52
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     min_samples_leaf 34
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 142 : Message :   test r2 score: 0.3992977208659255
2022-11-17T14:59:59CST : INFO : 

In [35]:
slug_rf.score

[0.5264056467634264, 0.39565984792823017]